In [ ]:
import pandas as pd 
import numpy as np

import csv
from progressbar import ProgressBar

import os, io, json
import multiprocessing
from pprint import pprint
import itertools
from nltk.corpus import stopwords
import gensim
from random import randint

from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

from gensim.models import FastText

import pickle 

from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
pbar = ProgressBar()

train_csv_path =  "/content/drive/MyDrive/Colab Notebooks/train2(clean_text).csv"
tokenizer_path = "/content/drive/MyDrive/Colab Notebooks/tokenize.json"
train_pickle ="/content/drive/MyDrive/Colab Notebooks/train.pickle"
embedin_dir="/content/drive/MyDrive/Colab Notebooks/fasttext/ft3_embed.model"
embed_matrix ="/content/drive/MyDrive/Colab Notebooks/embed_matrix_1.pickle"
labelencoder_path = "/content/drive/MyDrive/Colab Notebooks/labelencoder.pickle"
save_model_path = "/content/drive/MyDrive/Colab Notebooks/"
test_csv_path = "/content/drive/MyDrive/Colab Notebooks/test(text_clean).csv"

In [ ]:
df_test = pd.read_csv(test_csv_path,escapechar='\\',quoting=csv.QUOTE_NONE)

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
import tensorflow as tf
try: # detect TPUs
  tpu = None
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError: # detect GPUs
  #strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
  #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  strategy = tf.distribute.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0',), communication = CommunicationImplementation.AUTO
Number of accelerators:  1


In [ ]:
df = pd.read_csv(train_csv_path)
y_train = df["BROWSE_NODE_ID"]
del df
with open(labelencoder_path, 'rb') as handle:
  le2 = pickle.load(handle)
y_train = le2.fit_transform(y_train)
with open(train_pickle, 'rb') as handle:
  train_set = pickle.load(handle)
len(train_set)

1632277

In [ ]:
corpus = df['text_clean']
## create list of lists of unigrams
lst_corpus = []
for string in corpus:
   lst_words = string.split()
   lst_grams = [" ".join(lst_words[i:i+1]) 
               for i in range(0, len(lst_words), 1)]
   lst_corpus.append(lst_grams)
  

In [ ]:
#del corpus

In [ ]:
tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', 
                     oov_token="NaN", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(lst_corpus)
dic_vocabulary = tokenizer.word_index
## create sequence
lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)
## padding sequence
X_train = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=50, padding="post", truncating="post")


In [ ]:
len(X_train)
del lst_corpus

In [ ]:
import pickle 
with open(tokenizer_path, 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(train_pickle, 'rb') as handle:
  train_set = pickle.load(handle)
len(train_set)

1632277

In [ ]:

del tokenizer

In [ ]:
jso = tokenizer.to_json()
with io.open(tokenizer_path,'w', encoding='utf-8')  as f:
  f.write(json.dumps(jso, ensure_ascii=False))

In [ ]:
from keras_preprocessing.text import tokenizer_from_json
with open(tokenizer_path) as f:
  data = json.load(f)
  token_2 = tokenizer_from_json(data)

In [ ]:
del token_2

In [ ]:
text = tokenizer.sequences_to_texts([X_train[1]])
text

['imusti ultimate selfsufficiency handbook complete guide baking craft gardening preserving harvest raising animal nan NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN']

In [ ]:



cores = multiprocessing.cpu_count()
model_ted = FastText(lst_corpus, size=100, window=5, min_count=5, workers=cores,sg=1)
model_ted.save(embedin_dir)

In [ ]:

embed_model= FastText.load(embedin_dir)

In [ ]:
dic_vocabulary = token_2.word_index

In [ ]:
embed_model= FastText.load(embedin_dir)

In [ ]:
embeddings = np.zeros((len(dic_vocabulary)+1, 100))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embeddings[idx] =  embed_model[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:

with open(embed_matrix, 'wb') as handle:
  pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#LOAD EMBEDDINGS
with open(embed_matrix, 'rb') as handle:
  embeddings = pickle.load(handle)

In [ ]:
## code attention layer
def attention_layer(inputs, neurons):
    x = layers.Permute((2,1))(inputs)
    x = layers.Dense(neurons, activation="softmax")(x)
    x = layers.Permute((2,1), name="attention")(x)
    x = layers.multiply([inputs, x])
    return x

## input
x_in = layers.Input(shape=(50,))
## embedding
x = layers.Embedding(input_dim=embeddings.shape[0],  
                     output_dim=embeddings.shape[1], 
                     weights=[embeddings],
                     input_length=50, trainable=False)(x_in)
## apply attention
x = attention_layer(x, neurons=15)
## 2 layers of bidirectional lstm
x = layers.Bidirectional(layers.LSTM(units=30, dropout=0.2, 
                         return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(units=30, dropout=0.2))(x)
## final dense layers
x = layers.Dense(2048, activation='relu')(x)
y_out = layers.Dense(9919, activation='softmax')(x)
## compile
model = models.Model(x_in, y_out)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 100)           215697600 
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 60)            31440     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60)                21840     
_________________________________________________________________
dense (Dense)                (None, 2048)              124928    
_________________________________________________________________
dense_1 (Dense)              (None, 9919)              20324031  
Total params: 236,199,839
Trainable params: 20,502,239
Non-trainable params: 215,697,600
______________________________________

In [ ]:
# for tpu 
with strategy.scope():
  x_in = layers.Input(shape=(50,))
## embedding
  x = layers.Embedding(input_dim=embeddings.shape[0],  
                     output_dim=embeddings.shape[1], 
                     weights=[embeddings],
                     input_length=50, trainable=False)(x_in)
## apply attention
  x = attention_layer(x, neurons=15)
## 2 layers of bidirectional lstm
  x = layers.Bidirectional(layers.LSTM(units=30, dropout=0.2, 
                         return_sequences=True))(x)
  x = layers.Bidirectional(layers.LSTM(units=30, dropout=0.2))(x)
## final dense layers
  x = layers.Dense(2048, activation='relu')(x)
  y_out = layers.Dense(9919, activation='softmax')(x)
## compile
  model = models.Model(x_in, y_out)
  model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

  model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 100)           215697600 
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 60)            31440     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60)                21840     
_________________________________________________________________
dense (Dense)                (None, 2048)              124928    
_________________________________________________________________
dense_1 (Dense)              (None, 9919)              20324031  
Total params: 236,199,839
Trainable params: 20,502,239
Non-trainable params: 215,697,600
______________________________________

In [ ]:
y_train = y_train[:-7]
train_set = train_set[:-7]

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)


In [ ]:
model.load_weights("/content/drive/MyDrive/Colab Notebooks/modelsweights.h5")

[[1, 23, 4], [1, 2]]

In [ ]:
earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)
checkpoint = ModelCheckpoint(save_model_path,
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=4,
    verbose=0`,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)
callbacks = [earlystop, checkpoint,reduce_lr]


In [ ]:
training = model.fit(x=train_set, y=y_train, batch_size=256, 
                     epochs=1,  shuffle=True, verbose=1, validation_split=0.3,
                      callbacks = callbacks)

save_model(model,save_model_path)

In [ ]:
model.evaluate(train_set, y_train)

51009/51009 [==============================] - 377s 7ms/step - loss: 1.6553 - accuracy: 0.6423


[1.6553380489349365, 0.6423100829124451]

In [ ]:
def _save_parameters(model, save_folder):
    parameters = [self.input_shape, self.conv_filters ,self.conv_kernels, 
    self.conv_strides,self.latent_space_dim]
    save_path = os.path.join(save_folder + "parameters.pkl")
    with open(save_path,"wb") as f:
      pickle.dump(parameters, f)

def _save_weights(model,save_folder):
    save_path = os.path.join(save_folder + "weights.h5")
    model.save_weights(save_path)

def _create_folder_if_it_dosent_exist(folder):
    if not os.path.exists(folder):
      os.makedirs(folder)
def save_model(model, save_folder):
    _create_folder_if_it_dosent_exist(save_folder)
    #_save_parameters(model, save_folder)
    _save_weights(model,save_folder)



In [ ]:
save_model(model,save_model_path)

In [ ]:

le.classes_


array([      0,       1,       2, ..., 2847632, 2860317, 2893574])

In [ ]:
from keras_preprocessing.text import tokenizer_from_json
with open(tokenizer_path) as f:
  data = json.load(f)
  token_2 = tokenizer_from_json(data)
with open(labelencoder_path, 'rb') as handle:
  le = pickle.load( handle)
  

In [ ]:
df_test = pd.read_csv(test_csv_path)

In [ ]:
corpus = df_test['text_clean']
## create list of lists of unigrams
lst_corpus = []
for string in corpus:
   lst_words = string.split()
   lst_grams = [" ".join(lst_words[i:i+1]) 
               for i in range(0, len(lst_words), 1)]
   lst_corpus.append(lst_grams)
del corpus
#dic_vocabulary = token_2.word_index
## create sequence
lst_text2seq= token_2.texts_to_sequences(lst_corpus)
## padding sequence
X_test = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=50, padding="post", truncating="post")
del lst_text2seq


In [ ]:

del lst_corpus 
del token_2
del data

In [ ]:
X_test_a = X_test[:-775]
len(X_test_a)


110000

In [ ]:
X_test_b = X_test[-775:]
len(X_test_b)


775

In [ ]:
preds =[]
for i in range(0,110000,10000):
  
  logits = model.predict(X_test_a[i:i+10000])
  index = np.argmax(logits,axis=1)
  node = le.inverse_transform(index)
  preds = preds + node.tolist()
    
  

logits = model.predict(X_test_b)

print(len(X_test))
print(len(preds))

110775
110000


In [ ]:


index = np.argmax(logits,axis=1)
node = le.inverse_transform(index)
preds = preds + node.tolist()
print(len(X_test))
print(len(preds))
    

110775
110775


In [ ]:
model.load_weights("/content/drive/MyDrive/Colab Notebooks/modelsweights.h5")

In [ ]:
hr = [1,2,3]
hr = hr + hr
hr

[1, 2, 3, 1, 2, 3]

In [ ]:

df_test["BROWSE_NODE_ID"] = preds 
 
df_test = df_test[["PRODUCT_ID","BROWSE_NODE_ID"]]
df_test.to_csv("/content/drive/MyDrive/Colab Notebooks/submission_3.csv")

In [ ]:
df_test

,Unnamed: 0,PRODUCT_ID,text_clean
0,0,1,command command 3m small kitchen hook white de...
1,1,2,oneal oneal jump hardware jag unisexadult glov...
2,2,3,boelter brand nfl detroit lion portable party ...
3,3,4,panasonic panasonic single line kxts880mx cord...
4,4,5,zero zero baby girl 100 cotton innerwear bloom...
...,...,...,...
110770,110770,110771,generic aahna e mall oneblade hybrid trimmer s...
110771,110771,110772,grin health grin health n99 anti pollution reu...
110772,110772,110773,asian asian army pink ultra reusable respirato...
110773,110773,110774,intermarket im safe 3 ply nonwoven disposable ...
